In [1]:
from datetime import datetime, timedelta

def parse_date_publish(date_time_string):
    # Split the string into date, time, and timezone offset parts
    date_string, time_string = date_time_string.split()
    combined_string = f"{date_string} {time_string}"
    print('parse_date_publish', combined_string)
    # Parse the combined string into a datetime object
    parsed_datetime = datetime.strptime(combined_string, "%d-%m-%Y %H:%M")
    return parsed_datetime

# Input date and time string
date_time_string = "18-08-2023 15:00 GMT+7"
# Parse the date and time
parsed_datetime = parse_date_publish(date_time_string)
# Print the parsed datetime
print(parsed_datetime.strftime("%Y-%m-%d %H:%M:%S"))


parse_date_publish 18/08/2023 15:00
2023-08-18 15:00:00


In [19]:
import requests
def request_get(url, headers = {}, params = {}):
    # Make an HTTP GET request to the API endpoint using the requests library
    response = requests.request("GET", url, headers=headers, data=params)
    return response

url = "https://dantri.com.vn/api/newest/get-more-newest-article/638420457801630000/24/24.htm"
# create header fake browser chrome
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}
response = request_get(url, headers = headers, params = {})
print(response.content)

b'{"data":"\\r\\n\\r\\n    <article class=\\"article-list article-item\\" data-content-name=\\"category-timeline_page_{0}\\" data-content-piece=\\"category-timeline_page_{0}-position_{1}\\" data-content-target=\\"/tinh-yeu-gioi-tinh/gap-ban-trai-cung-vo-cu-di-sieu-thi-thai-do-cua-anh-khien-toi-dau-long-20240124215125716.htm\\" data-track-content=\\"\\">\\r\\n            <div class=\\"article-time\\">3h tr\xc6\xb0\xe1\xbb\x9bc</div>\\r\\n        <div class=\\"article-thumb\\">\\r\\n            <a href=\\"/tinh-yeu-gioi-tinh/gap-ban-trai-cung-vo-cu-di-sieu-thi-thai-do-cua-anh-khien-toi-dau-long-20240124215125716.htm\\"><img alt=\\"G\xe1\xba\xb7p b\xe1\xba\xa1n trai c\xc3\xb9ng v\xe1\xbb\xa3 c\xc5\xa9 \xc4\x91i si\xc3\xaau th\xe1\xbb\x8b, th\xc3\xa1i \xc4\x91\xe1\xbb\x99 c\xe1\xbb\xa7a anh khi\xe1\xba\xbfn t\xc3\xb4i \xc4\x91au l\xc3\xb2ng\\" height=\\"144\\" src=\\"https://cdnphoto.dantri.com.vn/eHirsRz95IJTRSXmnHNBI54tzXA=/zoom/324_216/2024/01/24/gap-ban-trai-dua-vo-cu-va-con-gai-di-sie

In [56]:
import html
import imp
import re

import json
from unittest import result
from bs4 import BeautifulSoup
from sympy import false

from datetime import datetime, timedelta
import time
import importlib

import crawl
importlib.reload(crawl)

import cl_crawl.helper as helper
importlib.reload(crawl.helper)

class CrawlDantri:
    baseUrl = 'https://dantri.com.vn'
    url = 'https://dantri.com.vn/api/newest/get-more-newest-article/{sessionId}/{offsetNext}/{offsetPrev}.htm'
    offsetCurrent = 12
    offsetPrev = 12
    isDupArticle = False
    isLastResult = False

    def __init__(self, is_update = False):
        self.is_update = is_update

    def get_page_detail(self, url, params = {}):
        url = self.baseUrl + url
        headers = {}
        response = request_get(url, params, headers)
        if response.status_code // 100 == 2:
            content_html = response.text
            print('content_html', content_html[:100])
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(content_html, 'html.parser')
            # Find the title element
            publishdate_elem = soup.find('time',  class_='author-time')
            content_detail_elem = soup.find('div', class_='singular-content')
            print('publishdate_elem', publishdate_elem)
            if publishdate_elem:
                publishDate = publishdate_elem.get('datetime')
                publishDate = publishDate.strip()
                print('publishDate', publishDate)
                publishDate = datetime.strptime(publishDate, "%Y-%m-%d %H:%M")
            else:
                publishDate = None

            content_detail = content_detail_elem.get_text() if content_detail_elem else None

            return {
                'date': publishDate,
                'content': content_detail
            }
        else:
            # If unsuccessful, print the status code and reason for failure
            print(f"Request failed with status code {response.status_code}: {response.reason}")
            return {}

    def crawl_html(self, url, params = {}):
        print('crawl_html', url, params)
        response = request_get(url, params)

        if response.status_code // 100 == 2:
            content_html = response.content
            # json parse
            content_json = json.loads(content_html)
            self.offsetCurrent = content_json['offset']
            self.offsetPrev = content_json['offset']
            html = content_json['data']
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Find the title element and check last result.
            title_elements = soup.select('.article-thumb a')

            if title_elements is None or title_elements is []:
                self.isLastResult = True

            for title_element in title_elements:
                title = title_element.find('img').get('alt')
                link = title_element['href']

                print("Title:", title)
                print("Link:", link)

                data_detail = self.get_page_detail(link, {})
                # print('data_detail', data_detail)
                item = {
                    'domain': self.baseUrl,
                    'title': title,
                    'url': link,
                    'date': data_detail['date'] if 'date' in data_detail else None,
                    'content': data_detail['content'] if 'content' in data_detail else None,
                }
                self.isDupArticle = helper.create_article(item, self.is_update)
        else:
            # If unsuccessful, print the status code and reason for failure
            print(f"Request failed with status code {response.status_code}: {response.reason}")
            return False

    def run(self, is_update = False, sessionId = ''):
        while (True):
            url_crawl = self.url.format(sessionId=sessionId, offsetNext=self.offsetCurrent, offsetPrev=self.offsetPrev)
            print('url_crawl', url_crawl, self.isLastResult)
            self.crawl_html(url_crawl, {})
            if (self.isDupArticle and not is_update) or self.isLastResult:
                break
            # set timeout before next request
            time.sleep(1)

obj = CrawlDantri()
obj.run(True, '638421102809130000')


url_crawl https://dantri.com.vn/api/newest/get-more-newest-article/638421102809130000/12/12.htm False
crawl_html https://dantri.com.vn/api/newest/get-more-newest-article/638421102809130000/12/12.htm {}
Title: Trẻ 2 tháng tuổi phải thở máy vì loại virus hay gặp mùa rét
Link: /suc-khoe/tre-2-thang-tuoi-phai-tho-may-vi-loai-virus-hay-gap-mua-ret-20240129072419732.htm
content_html <!DOCTYPE html>
<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="UTF-8">
publishdate_elem <time class="author-time" datetime="2024-01-29 07:30"> Thứ hai, 29/01/2024 - 07:30 </time>
publishDate 2024-01-29 07:30
Title: Giá vàng liệu có bứt phá trong tuần này?
Link: /kinh-doanh/gia-vang-lieu-co-but-pha-trong-tuan-nay-20240129004831760.htm
content_html <!DOCTYPE html>
<html lang="vi" xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="UTF-8">
publishdate_elem <time class="author-time" datetime="2024-01-29 07:27"> Thứ hai, 29/01/2024 - 07:27 </time>
publishDate 2024-01-29 07:27
Title: 

KeyboardInterrupt: 